In [1]:
import csv
import json
import pickle
import pandas as pd
from itertools import islice
from cyordereddict import OrderedDict
from collections import defaultdict
import re
import numpy as np
from fastai.text import *
import html
from sklearn.model_selection import train_test_split
from collections import Counter
import pickle
import ast
import multiprocessing

re1 = re.compile(r'  +')
chunksize = 100000

try:
    cpus = multiprocessing.cpu_count()
except NotImplementedError:
    cpus = 2   # arbitrary default


###File Names
TEST_NAME = "Fastai_basic_recipe"

DATASET_FILE_PATH = "./DataSet/yelp.csv"
WORD2VEC_FILE_PATH = "./DataSet/FBword2vec/wiki.en.vec"
TRAINING_DATA_PATH = "./DataSet/trainingData"
EXTENDED_DATASET_PATH = "../DataSet/review.json"

EMBEDDED_MATRIX_NAME = TEST_NAME+'_embedded_matrix.pkl'
TOKENIZED_SENTENCES_NAME = TEST_NAME+ "_tokenized_sentences.pkl"
WORD_INDEX_NAME = TEST_NAME+ "_word_index.pkl"
INDEXED_TOKENIZED_SENTENCES_NAME = TEST_NAME + "_indexed_tokenized_sentences.csv"

LABELS_NAME = 'labels.pkl'

DATASET_FILE_PATH = "../DataSet/yelp.csv/yelp.csv"
TRAINING_DATA_PATH = "../DataSet/trainingData"
EXTENDED_DATASET_PATH = "../DataSet/review.json"

BOS  =  'xbos'    # beginning-of-sentence tag 
FLD = 'xfld'  # data field tag
max_vocab = 60000
min_freq = 2

In [10]:
#Fast.ai tutorial code
def fixup(x):
    x = x.replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
        '<br />', "\n").replace('\\"', '"').replace('<unk>','u_n').replace(' @.@ ','.').replace(
        ' @-@ ','-').replace('\\', ' \\ ')
    return re1.sub(' ', html.unescape(x))

def get_texts(df, n_lbls=1):
    labels = df.iloc[:,range(n_lbls)].values.astype(np.int64)
    texts = f'\n{BOS} {FLD} 1 ' + df['text']
    for i in range(n_lbls+1, len(df.columns)): texts += f' {FLD} {i-n_lbls} ' + df[i].astype(str)
    texts = list(texts.apply(fixup).values)
    tok = Tokenizer().proc_all_mp(partition_by_cores(texts))
    return tok

def process_text(text):
    text = f"\n{BOS} {FLD} 1 " + fixup(text)
    return Tokenizer().proc_text(text)

In [3]:
def process_text(text):
    text = f"\n{BOS} {FLD} 1 " + fixup(text)
    return Tokenizer().proc_text(text)

In [ ]:
extended_dataset = pd.read_csv("Fastai_basic_recipe_tokenized_sentences.csv")#, chunksize=chunksize)
print("Finished loading")
train, test = train_test_split(extended_dataset, test_size=0.2)
print("Test split")
test.to_csv(INDEXED_TOKENIZED_SENTENCES_NAME+"_test",encoding='utf-8')
print("Saved test set")
train.to_csv(INDEXED_TOKENIZED_SENTENCES_NAME+"_train",encoding='utf-8')
print("Saved train set")
sentences = [ast.literal_eval(sentence) for sentence in train['text']]
print("Transformed strings to list")
Vocab = Counter(word for sentence in sentences for word in sentence)
print("Made vocab file")
with open('vocab.pkl', 'wb') as outputfile:
    pickle.dump(Vocab, outputfile)
print("Saved vocab")

Finished loading
Test split
Saved test set
Saved train set


In [11]:
train_dataset = pd.read_csv(INDEXED_TOKENIZED_SENTENCES_NAME+"_train", chunksize=chunksize)
Vocab = Counter()
progress = 0
for chunk in train_dataset:
    print(progress*chunksize)
    sentences = (ast.literal_eval(sentence) for sentence in chunk['text'])
    Vocab.update(word for sentence in sentences for word in sentence)
    progress+=1
print(Vocab)
with open('vocab.pkl', 'wb') as outputfile:
    pickle.dump(Vocab, outputfile)

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [19]:
itos = [o for o,c in Vocab.most_common(max_vocab) if c>min_freq]
itos.insert(0, '_pad_')
itos.insert(0, '_unk_')
stoi = defaultdict(lambda:0, {v:k for k,v in enumerate(itos)})
len(itos)
with open('itos.pkl', 'wb') as outputfile:
    pickle.dump(itos, outputfile)

In [30]:
def token2int(text, stoi_dict):
    tokens = (word for word in ast.literal_eval(text))
    return [stoi_dict[token] for token in tokens]

In [34]:
train_dataset = pd.read_csv(INDEXED_TOKENIZED_SENTENCES_NAME+"_train", chunksize=chunksize)
progress = 0
for chunk in train_dataset:
    print(progress*chunksize)
    chunk.drop(chunk.columns[chunk.columns.str.contains('unnamed',case = False)],axis = 1)
    chunk['text'] = chunk['text'].apply(token2int, stoi_dict=stoi)
    with open(TOKENIZED_SENTENCES_NAME+"_train_TO_INT", 'a', encoding="utf-8") as f:
        chunk.to_csv(f, header=(progress==0), encoding="utf-8")
    progress+=1

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000


In [38]:
test_dataset = pd.read_csv(INDEXED_TOKENIZED_SENTENCES_NAME+"_test", chunksize=chunksize)
progress = 0
for chunk in test_dataset:
    print(progress*chunksize)
    chunk.drop(chunk.columns[chunk.columns.str.contains('unnamed',case = False)],axis = 1)
    chunk['text'] = chunk['text'].apply(token2int, stoi_dict=stoi)
    with open(TOKENIZED_SENTENCES_NAME+"_test_TO_INT", 'a', encoding="utf-8") as f:
        chunk.to_csv(f, header=(progress==0), encoding="utf-8")
    progress+=1

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000


In [1]:
### WIKITEXT CONVERSION ###

In [2]:
em_sz,nh,nl = 400,1150,3
PRE_PATH = PATH/'models'/'wt103'
PRE_LM_PATH = PRE_PATH/'fwd_wt103.h5'
wgts = torch.load(PRE_LM_PATH, map_location=lambda storage, loc: storage)

In [3]:
### encoding mean to be used for unknown values
enc_wgts = to_np(wgts['0.encoder.weight'])
row_m = enc_wgts.mean(0)

NameError: name 'to_np' is not defined

In [ ]:
wiki_itos = pickle.load((PRE_PATH/'itos_wt103.pkl').open('rb'))
wiki_stoi = defaultdict(lambda:-1, {v:k for k,v in enumerate(wiki_itos)})

In [ ]:
### matching vocabulary
new_w = np.zeros((vs, em_sz), dtype=np.float32)
for i,w in enumerate(itos):
    r = wiki_stoi[w]
    new_w[i] = enc_wgts[r] if r>=0 else row_m

In [ ]:
### overwrite decoder module in torch
wgts['0.encoder.weight'] = T(new_w)
wgts['0.encoder_with_dropout.embed.weight'] = T(np.copy(new_w))
wgts['1.decoder.weight'] = T(np.copy(new_w))

In [ ]:
### Language Model ###

In [3]:
sentence_column_number = 9
trn_lm = pd.read_csv(TOKENIZED_SENTENCES_NAME+"_train_TO_INT", usecols=[sentence_column_number])
pool = multiprocessing.Pool(processes=cpus)
trn_lm = pool.map(eval, trn_lm['text'])

In [ ]:
trn_dl = LanguageModelLoader(np.concatenate(trn_lm), bs, bptt)
val_dl = LanguageModelLoader(np.concatenate(val_lm), bs, bptt)
md = LanguageModelData(PATH, 1, vs, trn_dl, val_dl, bs=bs, bptt=bptt)

In [ ]:
drops = np.array([0.25, 0.1, 0.2, 0.02, 0.15])*0.7

In [ ]:
learner= md.get_model(opt_fn, em_sz, nh, nl, 
    dropouti=drops[0], dropout=drops[1], wdrop=drops[2], dropoute=drops[3], dropouth=drops[4])

learner.metrics = [accuracy]
learner.freeze_to(-1)

In [ ]:
learner.model.load_state_dict(wgts)

In [ ]:
lr=1e-3
lrs = lr

In [ ]:
learner.fit(lrs/2, 1, wds=wd, use_clr=(32,2), cycle_len=1)